In [8]:
print('Hello')

Hello


In [9]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\End-to-end-medical-Chatbot\\research'

In [10]:
import os
os.chdir('../')

In [11]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\End-to-end-medical-Chatbot'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
# Extract the data from the pdf
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )

    docs = loader.load()

    return docs

In [14]:
extracted_data = load_pdf_file('Data/')

In [ ]:
extracted_data

In [16]:
# split the data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print('Length of text chunks', len(text_chunks))

Length of text chunks 5859


In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
# download the embeddings from hugging face

def download_hugging_face_emeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings = download_hugging_face_emeddings()

C:\Users\Sanish Bhagat\AppData\Local\Temp\ipykernel_16696\2254165220.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [21]:
query_result = embeddings.embed_query('Hello World')
print('Length', len(query_result))

Length 384


In [22]:
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


# pc.create_index(
#     name=index_name,
#     dimension=384, 
#     metric="cosine", 
#     spec=ServerlessSpec(
#         cloud="aws", 
#         region="us-east-1"
#     )
# ) 

In [26]:
# embed each chunk and upsert them into the pinecone index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

c:\Users\Sanish Bhagat\anaconda3\envs\medibot\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [27]:
# load existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [28]:
docsearch

In [29]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [30]:
retrieved_docs = retriever.invoke('What is Acne')

In [31]:
retrieved_docs

[Document(id='cad6c619-63a7-4a03-88e7-68bb8a499f5d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c95d5aef-d478-4d4e-98bb-dc1cf2da1f08', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='cf88bcd6-1986-41c6-9127-9c10346199e1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'Py

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-1.5-pro"
    google_api_key="AIzaSyB2dq3W_3R2N4lE7KN8MK1qVwk"
)

llm.invoke('Hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--11003a43-5ddc-422c-8c36-8fc3ac4392d0-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = """
You are a medical question-answering assistant.

- If the user greets you (e.g., "hi", "hello", "good morning") or uses polite phrases 
  (e.g., "thank you", "bye", "ok"), respond politely in a short, friendly way.

- If the user asks a medical-related question (health, symptoms, treatments, drugs, diseases, etc.), 
  answer using ONLY the provided medical data context.
  If the context does not contain the answer, respond with:
  "I don't know"

- If the user asks something unrelated to medicine (but not greetings/polite talk), respond with:
  "⚠️ This is a medical chatbot. Your query seems unrelated to medical topics. Please ask about health or medicine."

{context}
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({'input': 'What is Acne?'})
print(response['answer'])

Acne is the general name given to a skin disorder in which the sebaceous glands become inflamed.


In [44]:
response = rag_chain.invoke({'input': 'What is stats?'})
print(response['answer'])

I don't know
